In [3]:
import pandas as pd
# import transformers
import re
import os, os.path
import fnmatchs
import json
import collections
import glob

import matplotlib.pyplot as plt
import numpy as np

# import modules for web scrapping
import requests
from bs4 import BeautifulSoup


import pandas as pd

In [4]:
path='./medium_articles.csv'
df_medium=pd.read_csv(path)

def get_average_number_of_words(df):
  total_words=0;
  total_text=0;
  for text in df['text']:
    total_words+=len(list(text.split(" ")))
    total_text+=1

  return total_words/total_text  

In [5]:
df_medium= df_medium[df_medium['text'].str.len() <= 500]
df_medium=df_medium[0:1500]

In [6]:
persons = ["Mahatma_Gandhi","Nelson_Mandela","Abraham_Lincoln","Vladimir_Lenin","Jawaharlal_Nehru", "Nikola_Tesla","Albert_Einstein",\
          "Marie_Curie","Stephen_Hawking","Elon_Musk","Warren_Buffett","Jack_Ma","Oprah_Winfrey","Sachin_Tendulkar",\
          "Lionel_Messi","Milkha_Singh"]

df =  pd.DataFrame(columns = ['title', 'text'])

def FetchParagraphBetweenIds(id1,id2):
    hElem = soup.find("span", {'id': id1})
    endElem = soup.find('span', {'id': id2})
    cntns = list(soup.find_all())

    my_lst = []
    inBetween = False
    for tag in cntns:
        if tag == hElem:
            inBetween = True
        if inBetween == True and tag.name == 'p':
            my_lst.append(tag.get_text())
        if tag == endElem:
            inBetween = False
            break
    return "".join(my_lst)

for p in persons:
    url = f'https://en.wikipedia.org/wiki/{p}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    #print(response.status_code)
    soup = BeautifulSoup(response.content, 'html.parser')

    desired_ids = []
    for link in soup.find_all('span', attrs={'class':'mw-headline'}):
        if link.get('id') is not None:
            desired_ids.append(link.get('id'))

    id_to_paragraph = {}
    for i in range(len(desired_ids)-1):
        id_to_paragraph[desired_ids[i]] = FetchParagraphBetweenIds(desired_ids[i],desired_ids[i+1])
    id_to_paragraph_main = {}
    for k,v in id_to_paragraph.items():
        if len(v)>0:
            id_to_paragraph_main[k] = v
    df_new_row = pd.DataFrame(list(id_to_paragraph_main.items()), columns = ['title', 'text'])
    df = pd.concat([df, df_new_row])

df.reset_index(inplace=True)
df = df.dropna()
df['text'] = df['text'].astype('string')
df['title'] = df['title'].astype('string')


In [7]:
df=df[['text','title']]

In [18]:
print(f"The Shape of the WikiData data is -- {df.shape}")

The Shape of the WikiData data is -- (485, 2)


In [9]:
df_medium=df_medium[['text','title']]

In [13]:
frame=[df_medium,df]
combined_df=pd.concat(frame)

In [17]:
print(f"The Shape of the Combined data is -- {combined_df.shape}")

The Shape of the Combined data is -- (1985, 2)


In [16]:
df.to_csv('./medium_and_wikidata.csv',index=False)